In [18]:
import os

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage import(
    io, measure
)

# check for GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Current Device: {device}")

# Google Auth
# gauth = GoogleAuth()
# gauth.LocalWebserverAuth()

# drive = GoogleDrive(gauth)

PATH = 'data/INS1_BF/'

Current Device: cuda:0


### TODO:

- [x] Connect to google drive
- [x] Load tiffs into dataframe
- [x] Test train split
- [x] General linear regression model
- [ ] Train
- [ ] Validate





In [19]:
file_list = os.listdir(PATH)
print(file_list)

['0', '10000', '2000', '4000', '6000', '8000']


In [20]:
print(PATH+'0')

data/INS1_BF/0


In [23]:
temp = PATH+'0'
ds = datasets.ImageFolder.(temp, transform=transform)

SyntaxError: invalid syntax (<ipython-input-23-1ad34f71448d>, line 2)

In [25]:
from os import walk
for (dirpath, dirnames, filenames) in walk(PATH):
    print('Directory path: ', dirpath)
    print('Folder name: ', dirnames)

Directory path:  data/INS1_BF/
Folder name:  ['0', '10000', '2000', '4000', '6000', '8000']
Directory path:  data/INS1_BF/0
Folder name:  []
Directory path:  data/INS1_BF/10000
Folder name:  []
Directory path:  data/INS1_BF/2000
Folder name:  []
Directory path:  data/INS1_BF/4000
Folder name:  []
Directory path:  data/INS1_BF/6000
Folder name:  []
Directory path:  data/INS1_BF/8000
Folder name:  []


In [29]:
# Create Data

from torchvision import datasets, transforms

data = datasets.ImageFolder(root='data/INS1_BF', transform=transforms.ToTensor())

In [33]:
print(data.classes)

['0', '10000', '2000', '4000', '6000', '8000']


In [42]:
# Load data with data loader
from torch.utils.data import DataLoader
loader = DataLoader(data, shuffle=True)

In [59]:
# Create Model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(x.size(0),-1) # something broke right here dog
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

# Loss and Optimization
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [45]:
# Test Train Split
train_size = int(0.8*len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [46]:
print(len(train_dataset), len(test_dataset))

216 55


In [47]:
len(dataset)

271

In [48]:
type(train_dataset)

torch.utils.data.dataset.Subset

In [49]:
# Create test and train dataloaders

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=4,
                                         shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=4,
                                        shuffle=True, num_workers=2)

In [60]:
# Train CNN

for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader,0):
        inputs, labels = data # data is a tuple
        
        # zero parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print stats
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

RuntimeError: size mismatch, m1: [31808 x 497], m2: [400 x 120] at C:\cb\pytorch_1000000000000\work\aten\src\TH/generic/THTensorMath.cpp:41

In [63]:
dataset[1][0].shape

torch.Size([3, 2000, 2000])